In [1]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')


# Load environment variables from the .env file
load_dotenv()

True

In [12]:
def elastic_search(es_client, query, limit=3, filter=None, random=False):
    if random:
        search_query = {
          "size": 10,
          "query": {
            "function_score": {
              "functions": [
                {
                  "random_score": {}
                }
              ]
            }
          }
        }
    else:
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    # "filter": {
                    #     "term": {
                    #         "course": "data-engineering-zoomcamp"
                    #     }
                    # }
                }
            }
        }
    if filter is not None:
        search_query['query']['bool'].update({'filter': {"term": {"course": "machine-learning-zoomcamp"}}})

    response = es_client.search(index=index_name, body=search_query)
    
    return response['hits']['hits'][:limit]

def pretty(search_results):
    result_docs = []
    
    for hit in search_results:
        result_docs.append({hit['_score']: hit['_source']['question']})
    return result_docs


query = 'How do I execute a command in a running docker container?'
# query = 'PostgreSQL'
index_name = "course-questions"

res = elastic_search(es_client, query, random=True)
sample = res[0]['_source']
print(sample.keys())
print()
print(sample)

dict_keys(['title', 'tags', 'relief', 'positive_effects', 'flavours', 'avg_rating', 'num_ratings'])

{'title': 'Blood Diamond OG', 'tags': 'Indica', 'relief': 'chronic pain depression fatigue muscle spasms stress', 'positive_effects': 'body high relaxing sleepy', 'flavours': 'citrus herbal pine woody', 'avg_rating': 4.8, 'num_ratings': 50}
